In [1]:
# 📌 Celda 1 — Verificar versión de Python y librerías clave
!python --version

!pip show llama-index-core
!pip show llama-index-vector-stores-chroma
!pip show llama-index-embeddings-huggingface
!pip show chromadb
!pip show sentence-transformers
!pip show transformers
!pip show tokenizers
!pip show numpy


Python 3.10.18
Name: llama-index-core
Version: 0.12.20
Summary: Interface between LLMs and your data
Home-page: https://llamaindex.ai
Author: Jerry Liu
Author-email: jerry@llamaindex.ai
License: MIT
Location: c:\users\eduar\anaconda3\envs\vbd_01245_local\lib\site-packages
Requires: aiohttp, dataclasses-json, deprecated, dirtyjson, filetype, fsspec, httpx, nest-asyncio, networkx, nltk, numpy, pillow, pydantic, PyYAML, requests, SQLAlchemy, tenacity, tiktoken, tqdm, typing-extensions, typing-inspect, wrapt
Required-by: llama-index-embeddings-huggingface, llama-index-vector-stores-chroma
Name: llama-index-vector-stores-chroma
Version: 0.4.2
Summary: llama-index vector_stores chroma integration
Home-page: 
Author: 
Author-email: Your Name <you@example.com>
License-Expression: MIT
Location: c:\users\eduar\anaconda3\envs\vbd_01245_local\lib\site-packages
Requires: chromadb, llama-index-core
Required-by: 
Name: llama-index-embeddings-huggingface
Version: 0.5.2
Summary: llama-index embeddings 

In [2]:
# 📌 Celda 2 — Configuración de rutas locales para la persistencia del índice
import os

# 📂 Ruta raíz de trabajo local
BASE_DIR = r"C:\\batch001"

# 📂 Carpeta de almacenamiento para este test específico
STORAGE_DIR = os.path.join(BASE_DIR, "store_test_yaml2")

# 📂 Carpeta donde están los YAML de ontología
ONTOLOGY_DIR = os.path.join(BASE_DIR, "ontology")

# Crear carpeta de almacenamiento si no existe
os.makedirs(STORAGE_DIR, exist_ok=True)

print(f"📂 Carpeta base: {BASE_DIR}")
print(f"📂 Carpeta almacenamiento: {STORAGE_DIR}")
print(f"📂 Carpeta ontología: {ONTOLOGY_DIR}")


📂 Carpeta base: C:\\batch001
📂 Carpeta almacenamiento: C:\\batch001\store_test_yaml2
📂 Carpeta ontología: C:\\batch001\ontology


In [3]:
# 📌 Celda 3 — Inicialización de Chroma y StorageContext sin persist_dir inicial
import chromadb
from llama_index.vector_stores.chroma import ChromaVectorStore
from llama_index.core import StorageContext

# Inicializar cliente de Chroma persistente
chroma_client = chromadb.PersistentClient(path=os.path.join(STORAGE_DIR, "chroma"))
chroma_collection = chroma_client.get_or_create_collection("riesgos_collection")

# Crear VectorStore
vector_store = ChromaVectorStore(chroma_collection=chroma_collection)

# Crear StorageContext (sin persist_dir para evitar lectura de archivos inexistentes)
storage_context = StorageContext.from_defaults(vector_store=vector_store)

print("✅ StorageContext creado correctamente (sin persist_dir inicial).")



✅ StorageContext creado correctamente (sin persist_dir inicial).


In [4]:
# 📌 Celda 4 — Cargar modelo de embeddings
from llama_index.embeddings.huggingface import HuggingFaceEmbedding

# Modelo recomendado para semántica multilingüe (BAAI/bge-m3) o usar MiniLM si quieres más velocidad
embed_model = HuggingFaceEmbedding(model_name="BAAI/bge-m3")

print("✅ Modelo de embeddings cargado correctamente.")


C:\Users\eduar\anaconda3\envs\vbd_01245_local\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
C:\Users\eduar\anaconda3\envs\vbd_01245_local\lib\site-packages\huggingface_hub\file_download.py:945: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


✅ Modelo de embeddings cargado correctamente.


In [5]:
# 📌 Celda 5 — Preparar nodos desde el YAML de riesgos
# 📌 Celda 5 — Preparar nodos desde el YAML de riesgos (versión corregida con id_ explícito)

import yaml
from llama_index.core.schema import TextNode

# Ruta del archivo de riesgos
yaml_riesgos_path = os.path.join(ONTOLOGY_DIR, "01_catalogo_riesgos_v8.yaml")

if not os.path.exists(yaml_riesgos_path):
    raise FileNotFoundError(f"❌ No se encontró el archivo: {yaml_riesgos_path}")

# Cargar datos
with open(yaml_riesgos_path, "r", encoding="utf-8") as f:
    riesgos_data = yaml.safe_load(f)

nodes_riesgos = []

for item in riesgos_data:
    # Texto optimizado para búsquedas semánticas
    content = f"{item.get('nombre', '')}. {item.get('descripcion', '')}"

    # Metadatos ligeros (para VDB)
    metadata = {
        "id": item.get("id"),  # ID principal del riesgo (clave para VDB y GDB)
        "peligro_asociado": item.get("peligro_asociado"),
        "bowtie_id": item.get("bowtie_id"),
        "area": item.get("area"),
        "aplicabilidad": item.get("aplicabilidad"),
        "tags": ", ".join(item.get("tags", [])),
        # Flags de existencia (sin guardar listas largas en VDB)
        "tiene_causas": bool(item.get("causas_asociadas")),
        "tiene_controles_preventivos": bool(item.get("controles_preventivos_asociados")),
        "tiene_controles_criticos": bool(item.get("controles_criticos_asociados")),
        "tiene_controles_mitigacion": bool(item.get("controles_mitigacion_asociados")),
        "tiene_barreras_recuperacion": bool(item.get("barreras_recuperacion_asociadas")),
        "tiene_consecuencias": bool(item.get("consecuencias_asociadas")),
        "tiene_factores_degradacion": bool(item.get("factores_degradacion_asociados")),
        "tiene_factores_exposicion": bool(item.get("factores_exposicion_relacionados")),
        "tipo": "riesgo"
    }

    # ⚠️ Aquí la corrección: usar id_=item.get("id") para que no genere UUID
    node = TextNode(id_=item.get("id"), text=content, metadata=metadata)
    nodes_riesgos.append(node)

print(f"✅ Nodos de riesgos creados: {len(nodes_riesgos)}")
print("🔍 Ejemplo nodo:")
print("Texto:", nodes_riesgos[0].text)
print("Metadatos:", nodes_riesgos[0].metadata)


✅ Nodos de riesgos creados: 3
🔍 Ejemplo nodo:
Texto: Caída desde Altura. Este riesgo se refiere a la probabilidad de que una persona caiga desde un nivel superior a uno inferior, resultando en lesiones graves o fatales. Se materializa en situaciones como trabajos en andamios, plataformas elevadas, techos, escaleras verticales, o al transitar cerca de bordes desprotegidos (vanos, aberturas). La altura crítica suele definirse a partir de 1.8 metros, pero cualquier desnivel con potencial de lesión grave es relevante. Las consecuencias pueden variar desde fracturas y traumatismos severos hasta la muerte, además de daños a equipos o interrupción de la operación si la caída afecta infraestructura crítica.

Metadatos: {'id': 'R01', 'peligro_asociado': 'Trabajos en altura (≥1,8m o cerca de bordes sin protección)', 'bowtie_id': 'R01_BOW_CAIDA_V001', 'area': 'Todas las Áreas Operacionales', 'aplicabilidad': 'Identificación y evaluación de riesgos críticos', 'tags': 'caída de altura, impacto, seg

In [6]:
# 📌 Celda 6 — Persistencia siguiendo la lógica de test8
from llama_index.core import VectorStoreIndex

# Registrar nodos de riesgos en docstore
storage_context.docstore.add_documents(nodes_riesgos)

# Crear índice vacío
index = VectorStoreIndex(
    [],
    storage_context=storage_context,
    embed_model=embed_model
)

# Insertar nodos para generar embeddings y asociarlos a Chroma
index.insert_nodes(nodes_riesgos)

# Persistir todo el contenido (docstore, index_store, graph_store, Chroma)
storage_context.persist(persist_dir=STORAGE_DIR)

print(f"✅ Riesgos indexados y persistidos en: {STORAGE_DIR}")


✅ Riesgos indexados y persistidos en: C:\\batch001\store_test_yaml2


In [7]:
# 📌 Celda 7 — Procesar YAML de controles con flags booleanos (misma lógica que riesgos)
# 📌 Celda 7 — Procesar YAML de controles con flags booleanos y riesgo explícito (ID + nombre)

import yaml
from llama_index.core.schema import TextNode

# === Cargar catálogo de riesgos para mapear IDs -> nombres ===
yaml_riesgos = os.path.join(ONTOLOGY_DIR, "01_catalogo_riesgos_v8.yaml")
with open(yaml_riesgos, "r", encoding="utf-8") as f:
    riesgos = yaml.safe_load(f)

# Diccionario auxiliar: { "R01": "Caída de altura", "R02": "Caída de objetos", ... }
mapa_riesgos = {item["id"]: item["nombre"] for item in riesgos}

# === Cargar catálogo de controles ===
yaml_controles = os.path.join(ONTOLOGY_DIR, "02_catalogo_controles_v6.yaml")
with open(yaml_controles, "r", encoding="utf-8") as f:
    controles = yaml.safe_load(f)

# Campos explícitamente permitidos como metadatos (no duplicamos 'id')
campos_permitidos = {
    "tipo_control", "es_critico", "clase_implementacion", "aplicabilidad",
    "tags", "area", "registro_aplicable_a"
}

# Relaciones que se convierten en flags booleanos
relaciones_flags = {
    "riesgo_asociado": "tiene_riesgo_asociado",
    "causas_que_previene": "tiene_causas_que_previene",
    "consecuencias_que_mitiga": "tiene_consecuencias_que_mitiga",
    "factores_degradacion_que_lo_afectan": "tiene_factores_degradacion"
}

nodes_controles = []
for item in controles:
    content = f"{item.get('nombre', '')}. {item.get('descripcion', '')}"

    metadata = {}
    for k in campos_permitidos:
        if k == "tags":
            tags = item.get("tags", [])
            metadata["tags"] = ", ".join(tags) if isinstance(tags, list) else str(tags)
        elif k == "registro_aplicable_a":
            lista = item.get(k, [])
            metadata[k] = ", ".join(lista) if isinstance(lista, list) else str(lista)
        elif k == "es_critico":
            metadata[k] = bool(item.get(k))
        else:
            metadata[k] = item.get(k)

    # Flags booleanos
    for campo_yaml, campo_flag in relaciones_flags.items():
        metadata[campo_flag] = bool(item.get(campo_yaml))

    # === Asociación explícita con riesgo (ID + nombre) ===
    riesgo_id = item.get("riesgo_asociado")
    if riesgo_id:
        metadata["riesgo_asociado_id"] = riesgo_id
        metadata["riesgo_asociado_nombre"] = mapa_riesgos.get(riesgo_id, "Desconocido")

    # Crear nodo
    node = TextNode(text=content, metadata=metadata, id_=item.get("id"))
    nodes_controles.append(node)

print(f"✔️ Nodos de control procesados: {len(nodes_controles)}")
print("🔍 Ejemplo de nodo de control:\n")
print("Texto:", nodes_controles[0].text)
print("Metadatos:", nodes_controles[0].metadata)




✔️ Nodos de control procesados: 48
🔍 Ejemplo de nodo de control:

Texto: Sistema Personal de Detención de Caídas (SPDC). Implementación y uso obligatorio de arneses de cuerpo completo, líneas de vida con absorbedor de energía y puntos de anclaje certificados. Este sistema debe ser inspeccionado antes de cada uso, garantizar su compatibilidad entre componentes, y ser ajustado adecuadamente por el usuario. Su función principal es detener una caída de forma segura, minimizando las fuerzas de impacto sobre el cuerpo y evitando el contacto con el nivel inferior. Es el último recurso personal para prevenir la caída libre.

Metadatos: {'clase_implementacion': 'EPP', 'registro_aplicable_a': 'incidente, observacion', 'tags': 'SPDC, arnés, línea de vida, detención de caída, EPP, seguridad en altura, inspección pre-uso, anclaje certificado, fuerza de impacto, prevención de caída libre', 'area': 'Todas las Áreas Operacionales', 'aplicabilidad': 'Prevención de incidentes', 'es_critico': True, 'tipo

In [8]:
# 📌 Celda 8 — Insertar y persistir nodos de controles en la misma carpeta que riesgos (flujo seguro)

from llama_index.core import VectorStoreIndex

# 1️⃣ Registrar nodos de controles en docstore
for node in nodes_controles:
    index.docstore.add_documents([node])

# 2️⃣ Insertar nodos de controles en Chroma (generar embeddings)
index = VectorStoreIndex(
    list(index.docstore.docs.values()),  # todos los nodos actuales (riesgos + controles)
    storage_context=storage_context,
    embed_model=embed_model,
    store_nodes_in_index=True
)

# 3️⃣ Persistir todos los cambios (docstore, index_store y Chroma)
storage_context.persist(persist_dir=STORAGE_DIR)

# 4️⃣ Verificar
print(f"✅ Controles insertados y persistidos en: {STORAGE_DIR}")
print(f"📦 Total de nodos en docstore ahora: {len(index.docstore.docs)}")



Add of existing embedding ID: R01
Add of existing embedding ID: R02
Add of existing embedding ID: R03
Insert of existing embedding ID: R01
Insert of existing embedding ID: R02
Insert of existing embedding ID: R03


✅ Controles insertados y persistidos en: C:\\batch001\store_test_yaml2
📦 Total de nodos en docstore ahora: 51


In [1]:
# 📌 Celda A - Reinicio - Configuración inicial y verificación de entorno
import sys
import platform
import pkg_resources

print(f"Python {platform.python_version()}")

# Versión de librerías clave
for pkg in ["llama-index-core", "llama-index-vector-stores-chroma", 
            "llama-index-embeddings-huggingface", "chromadb"]:
    try:
        version = pkg_resources.get_distribution(pkg).version
        print(f"{pkg}: {version}")
    except pkg_resources.DistributionNotFound:
        print(f"{pkg}: ❌ No instalado")


C:\Users\eduar\AppData\Local\Temp\ipykernel_18668\2707857213.py:4: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  import pkg_resources


Python 3.10.18
llama-index-core: 0.12.20
llama-index-vector-stores-chroma: 0.4.2
llama-index-embeddings-huggingface: 0.5.2
chromadb: 0.5.23


In [2]:
# 📌 Celda B - Reinicio -  Restaurar almacenamiento existente
# 📌 Celda B - Reinicio - Restaurar almacenamiento existente
# 📌 Restaurar índice usando siempre el último registrado en index_store.json

import os, json
import chromadb
from llama_index.core import StorageContext, load_index_from_storage
from llama_index.embeddings.huggingface import HuggingFaceEmbedding
from llama_index.vector_stores.chroma import ChromaVectorStore

# ⚠️ Rutas fijas de tu proyecto
BASE_DIR = r"C:\batch001"
STORAGE_DIR = os.path.join(BASE_DIR, "store_test_yaml2")
ONTOLOGY_DIR = os.path.join(BASE_DIR, "ontology")  # ✅ agregado

# Cliente y colección de Chroma
chroma_client = chromadb.PersistentClient(path=os.path.join(STORAGE_DIR, "chroma"))
vector_store = ChromaVectorStore(
    chroma_collection=chroma_client.get_or_create_collection("riesgos_collection")
)

# Restaurar StorageContext
storage_context = StorageContext.from_defaults(
    vector_store=vector_store,
    persist_dir=STORAGE_DIR
)

# Embeddings
embed_model = HuggingFaceEmbedding(model_name="BAAI/bge-m3")

# 📌 Leer index_store.json y tomar el último índice
index_store_path = os.path.join(STORAGE_DIR, "index_store.json")
with open(index_store_path, "r", encoding="utf-8") as f:
    index_store_data = json.load(f)

index_ids = list(index_store_data["index_store/data"].keys())
print("📂 Índices encontrados en index_store.json:", index_ids)

if not index_ids:
    raise ValueError("❌ No se encontraron índices en index_store.json")

# Seleccionar siempre el último índice
last_index_id = index_ids[-1]
print(f"📌 Usando el último índice: {last_index_id}")

# Restaurar ese índice
index = load_index_from_storage(
    storage_context=storage_context,
    index_id=last_index_id,
    embed_model=embed_model
)

print(f"✅ Contexto e índice restaurados desde: {STORAGE_DIR}")




C:\Users\eduar\anaconda3\envs\vbd_01245_local\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
C:\Users\eduar\anaconda3\envs\vbd_01245_local\lib\site-packages\huggingface_hub\file_download.py:945: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


📂 Índices encontrados en index_store.json: ['5fef50fb-2ec8-4c24-ac8d-68fc3de1834d', '39acc04d-8ea2-4ba0-9d4b-a3a716589d6a']
📌 Usando el último índice: 39acc04d-8ea2-4ba0-9d4b-a3a716589d6a
✅ Contexto e índice restaurados desde: C:\batch001\store_test_yaml2


In [3]:
# 📌 Celda C — Verificación de sincronización después de restaurar el índice

# 1️⃣ Total de nodos en docstore
docstore_ids = set(index.docstore.docs.keys())
print(f"📚 Total de nodos en docstore: {len(docstore_ids)}")

# 2️⃣ Total de embeddings en Chroma
chroma_data = vector_store._collection.get(limit=999999)  # obtener todos
chroma_ids = set(chroma_data["ids"])
print(f"🟦 Total de embeddings en Chroma: {len(chroma_ids)}")

# 3️⃣ Comparación de IDs
faltan_en_chroma = docstore_ids - chroma_ids
faltan_en_docstore = chroma_ids - docstore_ids

if not faltan_en_chroma and not faltan_en_docstore:
    print("✅ Docstore y Chroma están sincronizados (mismos IDs).")
else:
    print("⚠️ Desincronización detectada:")
    if faltan_en_chroma:
        print(f"   → IDs en docstore pero no en Chroma: {faltan_en_chroma}")
    if faltan_en_docstore:
        print(f"   → IDs en Chroma pero no en docstore: {faltan_en_docstore}")

# 4️⃣ Mostrar un ejemplo de nodo restaurado
if docstore_ids:
    ejemplo_id = list(docstore_ids)[0]
    ejemplo_node = index.docstore.docs[ejemplo_id]
    print("\n🔍 Ejemplo de nodo restaurado:")
    print("ID:", ejemplo_node.node_id)
    print("Texto:", ejemplo_node.text[:150], "...")
    print("Metadatos:", ejemplo_node.metadata)


📚 Total de nodos en docstore: 51
🟦 Total de embeddings en Chroma: 51
✅ Docstore y Chroma están sincronizados (mismos IDs).

🔍 Ejemplo de nodo restaurado:
ID: R02CC06
Texto: Plan de Izaje y Maniobras de Carga Críticas (APROBADO). Documento formal y detallado que describe paso a paso cómo se realizará cada maniobra de izaje ...
Metadatos: {'clase_implementacion': 'administrativo', 'registro_aplicable_a': 'incidente, observacion', 'tags': 'plan de izaje, maniobras de carga, planificación, seguridad operacional, procedimiento, aprobación', 'area': 'Todas las Áreas Operacionales', 'aplicabilidad': 'Prevención de incidentes', 'es_critico': True, 'tipo_control': 'preventivo', 'tiene_riesgo_asociado': True, 'tiene_causas_que_previene': True, 'tiene_consecuencias_que_mitiga': False, 'tiene_factores_degradacion': True, 'riesgo_asociado_id': 'R02', 'riesgo_asociado_nombre': 'Caída de Objetos'}


In [4]:
# ✅ Celda 9 — Crear nodos de causas con flags booleanos para relaciones
# 📌 Celda 9 — Crear nodos de causas desde YAML con riesgo explícito (ID + nombre)

import yaml
from llama_index.core.schema import TextNode

# === Cargar catálogo de riesgos para mapear IDs -> nombres ===
yaml_riesgos = os.path.join(ONTOLOGY_DIR, "01_catalogo_riesgos_v8.yaml")
with open(yaml_riesgos, "r", encoding="utf-8") as f:
    riesgos = yaml.safe_load(f)

mapa_riesgos = {item["id"]: item["nombre"] for item in riesgos}

# === Cargar catálogo de causas ===
yaml_causas = os.path.join(ONTOLOGY_DIR, "03_catalogo_causas_v4.yaml")
with open(yaml_causas, "r", encoding="utf-8") as f:
    causas = yaml.safe_load(f)

# Campos explícitamente permitidos como metadatos (no duplicamos 'id')
campos_permitidos_causas = {
    "tipo_causa", "aplicabilidad", "tags", "area", "registro_aplicable_a"
}

# Relaciones que se convierten en flags booleanos
relaciones_flags_causas = {
    "riesgo_asociado": "tiene_riesgo_asociado",
    "controles_que_previenen": "tiene_controles_que_previenen",
    "eventos_que_desencadena": "tiene_eventos_desencadena"
}

nodes_causas = []
for item in causas:
    content = f"{item.get('nombre', '')}. {item.get('descripcion', '')}"
    metadata = {}

    # Metadatos base
    for k in campos_permitidos_causas:
        if k == "tags":
            tags = item.get("tags", [])
            metadata[k] = ", ".join(tags) if isinstance(tags, list) else str(tags)
        elif k == "registro_aplicable_a":
            lista = item.get(k, [])
            metadata[k] = ", ".join(lista) if isinstance(lista, list) else str(lista)
        else:
            metadata[k] = item.get(k)

    # Flags booleanos (relaciones)
    for campo_yaml, campo_flag in relaciones_flags_causas.items():
        metadata[campo_flag] = bool(item.get(campo_yaml))

    # === Asociación explícita con riesgo (ID + nombre) ===
    riesgo_id = item.get("riesgo_asociado")
    if riesgo_id:
        metadata["riesgo_asociado_id"] = riesgo_id
        metadata["riesgo_asociado_nombre"] = mapa_riesgos.get(riesgo_id, "Desconocido")

    # Crear nodo con ID estable
    node = TextNode(id_=item.get("id"), text=content, metadata=metadata)
    nodes_causas.append(node)

print(f"✔️ Nodos de causas procesados: {len(nodes_causas)}")
print("🔍 Ejemplo de nodo de causa:\n")
print("Texto:", nodes_causas[0].text)
print("Metadatos:", nodes_causas[0].metadata)



✔️ Nodos de causas procesados: 18
🔍 Ejemplo de nodo de causa:

Texto: Bordes o aberturas sin protección. Ausencia total o parcial de sistemas de protección perimetral (barandas, rodapiés, cubiertas) en bordes de plataformas, vanos, aberturas en el piso, o desniveles significativos. Esto permite que una persona, herramienta o material caiga sin contención. Puede variar en la altura del desnivel, el tipo de borde (recto, curvo), la presencia o ausencia de señalización, y el tipo de actividad que se realiza cerca del borde. Ejemplos incluyen la falta de barandas en una plataforma de trabajo, una abertura sin cubrir en un piso, o un desnivel sin rodapiés en una pasarela.

Metadatos: {'tipo_causa': 'Fallas de infraestructura', 'aplicabilidad': 'Prevención de incidentes', 'area': 'Todas las Áreas Operacionales', 'tags': 'seguridad perimetral, barandas, rodapiés, infraestructura, diseño, protección de borde, apertura, vano, desnivel, contención', 'registro_aplicable_a': 'incidente, observacio

In [5]:
# 📌 Celda 10 — Insertar y persistir nodos de causas

# 📌 Celda 10 — Insertar y persistir nodos de causas en la misma carpeta que riesgos y controles (flujo seguro)

from llama_index.core import VectorStoreIndex

# 1️⃣ Registrar nodos de causas en docstore
for node in nodes_causas:
    index.docstore.add_documents([node])

# 2️⃣ Insertar nodos de causas en Chroma (generar embeddings)
index = VectorStoreIndex(
    list(index.docstore.docs.values()),  # todos los nodos actuales (riesgos + controles + causas)
    storage_context=storage_context,
    embed_model=embed_model,
    store_nodes_in_index=True
)

# 3️⃣ Persistir todos los cambios (docstore, index_store y Chroma)
storage_context.persist(persist_dir=STORAGE_DIR)

# 4️⃣ Verificar
print(f"✅ Causas insertadas y persistidas en: {STORAGE_DIR}")
print(f"📦 Total de nodos en docstore ahora: {len(index.docstore.docs)}")



Add of existing embedding ID: R01
Add of existing embedding ID: R02
Add of existing embedding ID: R03
Insert of existing embedding ID: R01
Insert of existing embedding ID: R02
Insert of existing embedding ID: R03
Insert of existing embedding ID: R01CC01
Insert of existing embedding ID: R01CC02
Insert of existing embedding ID: R01CC03
Insert of existing embedding ID: R01CC04
Insert of existing embedding ID: R01CC05
Insert of existing embedding ID: R01CC06
Insert of existing embedding ID: R01CP01
Insert of existing embedding ID: R01CP02
Insert of existing embedding ID: R01CP03
Insert of existing embedding ID: R01CP04
Insert of existing embedding ID: R01CP05
Insert of existing embedding ID: R01CP06
Insert of existing embedding ID: R01CM01
Insert of existing embedding ID: R01CM02
Insert of existing embedding ID: R01CM03
Insert of existing embedding ID: R01CM04
Insert of existing embedding ID: R02CC01
Insert of existing embedding ID: R02CC02
Insert of existing embedding ID: R02CC03
Insert o

✅ Causas insertadas y persistidas en: C:\batch001\store_test_yaml2
📦 Total de nodos en docstore ahora: 69


In [10]:
# 📌 Celda 11 — Verificación de nodos de causas en docstore.json

import json
import os

docstore_path = os.path.join(persist_dir, "docstore.json")

# 1️⃣ Cargar docstore desde disco
with open(docstore_path, "r", encoding="utf-8") as f:
    docstore_data = json.load(f)

# 2️⃣ Filtrar solo nodos de causas (IDs que empiezan con R01CA, R02CA, R03CA)
causas_ids = [k for k in docstore_data["docstore/data"].keys() if "CA" in k]
print(f"📄 Total de causas encontradas en docstore: {len(causas_ids)}")

# 3️⃣ Mostrar un ejemplo con metadatos
if causas_ids:
    ejemplo_id = causas_ids[0]
    ejemplo_data = docstore_data["docstore/data"][ejemplo_id]
    print(f"\n🔍 Ejemplo de nodo de causa guardado en docstore ({ejemplo_id}):")
    print("Texto:", ejemplo_data["__data__"]["text"])
    print("Metadatos:", ejemplo_data["__data__"]["metadata"])
else:
    print("⚠️ No se encontraron nodos de causas en docstore.json")


📄 Total de causas encontradas en docstore: 18

🔍 Ejemplo de nodo de causa guardado en docstore (R01CA01):
Texto: Bordes o aberturas sin protección. Ausencia total o parcial de sistemas de protección perimetral (barandas, rodapiés, cubiertas) en bordes de plataformas, vanos, aberturas en el piso, o desniveles significativos. Esto permite que una persona, herramienta o material caiga sin contención. Puede variar en la altura del desnivel, el tipo de borde (recto, curvo), la presencia o ausencia de señalización, y el tipo de actividad que se realiza cerca del borde. Ejemplos incluyen la falta de barandas en una plataforma de trabajo, una abertura sin cubrir en un piso, o un desnivel sin rodapiés en una pasarela.

Metadatos: {'area': 'Todas las Áreas Operacionales', 'registro_aplicable_a': 'incidente, observacion', 'tags': 'seguridad perimetral, barandas, rodapiés, infraestructura, diseño, protección de borde, apertura, vano, desnivel, contención', 'tipo_causa': 'Fallas de infraestructura'

In [5]:
# 📌 Celda 12 — Crear nodos de consecuencias desde YAML
# 📌 Celda 12 — Crear nodos de consecuencias desde YAML (con severidad y flags booleanos para relaciones)

import yaml
from llama_index.core.schema import TextNode

yaml_consecuencias = "ontology/04_catalogo_consecuencias_v6.yaml"

with open(yaml_consecuencias, "r", encoding="utf-8") as f:
    consecuencias = yaml.safe_load(f)

# Campos base para metadatos (no incluyen IDs de relaciones)
campos_permitidos_consecuencias = {
    "tipo_consecuencia", "aplicabilidad", "tags", "area", "registro_aplicable_a", "severidad_consecuencia"
}

# Relaciones a convertir en flags booleanos
relaciones_flags_consecuencias = {
    "riesgo_asociado": "tiene_riesgo_asociado",
    "controles_mitigacion_asociados": "tiene_controles_mitigacion",
    "barreras_recuperacion_asociadas": "tiene_barreras_recuperacion"
}

nodes_consecuencias = []
for item in consecuencias:
    # Texto principal
    content = f"{item.get('nombre', '')}. {item.get('descripcion', '')}"
    metadata = {}

    # Procesar campos permitidos
    for k in campos_permitidos_consecuencias:
        if k == "tags":
            tags = item.get("tags", [])
            metadata[k] = ", ".join(tags) if isinstance(tags, list) else str(tags)
        elif k == "registro_aplicable_a":
            lista = item.get(k, [])
            metadata[k] = ", ".join(lista) if isinstance(lista, list) else str(lista)
        else:
            metadata[k] = item.get(k)

    # Procesar relaciones como flags booleanos (sin guardar IDs)
    for campo_yaml, campo_flag in relaciones_flags_consecuencias.items():
        metadata[campo_flag] = bool(item.get(campo_yaml))

    # Crear nodo
    node = TextNode(id_=item.get("id", None), text=content, metadata=metadata)
    nodes_consecuencias.append(node)

print(f"✔️ Nodos de consecuencias procesados: {len(nodes_consecuencias)}")
print("🔍 Ejemplo de nodo de consecuencia:")
print("Texto:", nodes_consecuencias[0].text)
print("Metadatos:", nodes_consecuencias[0].metadata)




✔️ Nodos de consecuencias procesados: 18
🔍 Ejemplo de nodo de consecuencia:
Texto: Lesiones graves. Fracturas óseas múltiples, traumatismos craneoencefálicos y contusiones profundas que requieren atención médica de urgencia. Estas lesiones suelen derivar en hospitalización prolongada, intervenciones quirúrgicas y tiempo de inactividad laboral significativo. El impacto puede incluir secuelas a largo plazo como limitaciones de movilidad o déficit neurológico. 
Metadatos: {'registro_aplicable_a': 'incidente', 'tipo_consecuencia': 'Salud_Persona', 'area': 'Todas las Áreas Operacionales', 'tags': 'gravedad, trauma, incapacidad, lesiones', 'aplicabilidad': 'Evaluación de consecuencias', 'severidad_consecuencia': 'ALTA', 'tiene_riesgo_asociado': True, 'tiene_controles_mitigacion': True, 'tiene_barreras_recuperacion': True}


In [15]:
# 📌 Celda 13 — Insertar consecuencias y persistir en Chroma y docstore.json (flujo seguro)
# 📌 Celda 13 — Insertar nodos de consecuencias en Chroma y docstore con verificación real
# 📌 Celda 13 — Inserción segura de consecuencias (comparación directa contra Chroma)

from llama_index.core import VectorStoreIndex
import json
import os

# 1️⃣ Extraer todos los IDs de consecuencias desde la lista de nodos
ids_consecuencias = [n.node_id for n in nodes_consecuencias]

# 2️⃣ Consultar a Chroma SOLO esos IDs
chroma_check = storage_context.vector_store._collection.get(
    ids=ids_consecuencias,
    where=None,
    limit=999999
)

ids_consecuencias_chroma = set(chroma_check["ids"])
print(f"📦 Consecuencias ya en Chroma: {len(ids_consecuencias_chroma)}/{len(ids_consecuencias)}")

# 3️⃣ Consultar al docstore SOLO esos IDs
ids_docstore = set(index.docstore.docs.keys())
ids_consecuencias_docstore = {id_ for id_ in ids_consecuencias if id_ in ids_docstore}
print(f"📚 Consecuencias ya en docstore: {len(ids_consecuencias_docstore)}/{len(ids_consecuencias)}")

# 4️⃣ Filtrar faltantes en Chroma y docstore
nodos_faltan_chroma = [n for n in nodes_consecuencias if n.node_id not in ids_consecuencias_chroma]
nodos_faltan_docstore = [n for n in nodes_consecuencias if n.node_id not in ids_consecuencias_docstore]

print(f"🆕 Nodos de consecuencias a insertar en Chroma: {len(nodos_faltan_chroma)}")
print(f"🆕 Nodos de consecuencias a insertar en docstore: {len(nodos_faltan_docstore)}")

# 5️⃣ Insertar en Chroma si faltan
if nodos_faltan_chroma:
    _ = VectorStoreIndex(
        nodos_faltan_chroma,
        storage_context=storage_context,
        embed_model=embed_model,
        store_nodes_in_index=True
    )
    print("✅ Consecuencias insertadas en Chroma.")

# 6️⃣ Insertar en docstore si faltan
if nodos_faltan_docstore:
    for node in nodos_faltan_docstore:
        index.docstore.add_documents([node])
    print("✅ Consecuencias insertadas en docstore.")

# 7️⃣ Persistir cambios
storage_context.persist()

# 8️⃣ Guardar manualmente el docstore.json
docstore_path = os.path.join(persist_dir, "docstore.json")
with open(docstore_path, "w", encoding="utf-8") as f:
    json.dump(index.docstore.to_dict(), f, ensure_ascii=False, indent=2)

print(f"💾 Persistencia completada. docstore.json actualizado en: {docstore_path}")



📦 Consecuencias ya en Chroma: 18/18
📚 Consecuencias ya en docstore: 18/18
🆕 Nodos de consecuencias a insertar en Chroma: 0
🆕 Nodos de consecuencias a insertar en docstore: 0
💾 Persistencia completada. docstore.json actualizado en: store_test_yaml\docstore.json


In [18]:
# 📌 Celda — Revisar IDs en Chroma para un tipo específico (ej. Consecuencias)

# Obtener TODOS los registros actuales en Chroma
chroma_data = storage_context.vector_store._collection.get(
    ids=None,
    limit=999999,  # evitar cortes
    include=["metadatas", "documents"]
)

# Lista completa de IDs en Chroma
ids_chroma = set(chroma_data["ids"])
print(f"📦 Total de IDs en Chroma: {len(ids_chroma)}")

# Filtrar solo los que corresponden a consecuencias (patrón 'CON' en el ID)
ids_consecuencias_chroma = sorted([id_ for id_ in ids_chroma if "CON" in id_])
print(f"📦 Consecuencias en Chroma: {len(ids_consecuencias_chroma)}")
print(ids_consecuencias_chroma)


📦 Total de IDs en Chroma: 87
📦 Consecuencias en Chroma: 18
['R01CON01', 'R01CON02', 'R01CON03', 'R01CON04', 'R01CON05', 'R01CON06', 'R01CON07', 'R02CON01', 'R02CON02', 'R02CON03', 'R02CON04', 'R02CON05', 'R03CON01', 'R03CON02', 'R03CON03', 'R03CON04', 'R03CON05', 'R03CON06']
